In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame
from ipywidgets import FloatProgress
from time import sleep
from IPython.display import display
import re
import pickle
from urlextract import URLExtract
import json
import sqlite3
import datetime
from tqdm import tqdm

In [2]:
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}

## Mens Clothing

In [30]:
men_clothings_link=[]
for i in range(1,7):
    try:
        result=requests.get(f"https://apparels.azanisport.com/collections/mens-clothing?page={i}",headers=headers)
        get=result.content
        data=BeautifulSoup(get,'lxml')
        prod=data.find_all('div',{'class':'Grid__Cell 1/2--phone 1/2--tablet-and-up 1/3--desk'})
        for j in prod:
            if(j.find('button').find('span').text !='SOLD OUT'):
                men_clothings_link.append('https://apparels.azanisport.com'+j.find('a').attrs['href'])
    except:
        pass 

In [31]:
len(men_clothings_link)

64

In [32]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
Gender=[]
Description=[]
Cat_list=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [33]:
for i in tqdm(men_clothings_link):
    try:
        result=requests.get(i,headers=headers)
        get=result.content
        soup=BeautifulSoup(get,'lxml')
        js=json.loads(soup.find('script',{'type':'application/ld+json'}).contents[0])
        product_name=js['name']
        product_brand=js['brand']
        product_link=i
        product_cat=js['category']
        product_des=js['description']
        product_mrp=soup.find('span',{'class':'ProductMeta__Price Price Price--compareAt Text--subdued u-h4'}).text
        product_mrp=int(''.join(product_mrp[4:].split(',')))
        product_price=int(float(js['offers']['price']))
        primary_link=js['image']['url']
        images=soup.find_all('div',{'class':'Product__SlideItem Product__SlideItem--image Carousel__Cell'})
        img=[]
        for k in images:
            img.append(k.find('img').attrs['data-src'].split('{width}')[0]+'1024x1024.jpg')
        secondary_link=','.join(img)
        size=soup.find('ul',{'class':'SizeSwatchList HorizontalList HorizontalList--spacingTight'}).text.lstrip().rstrip().split('\n\n\n')
        size_avl=','.join(size)
    
    except:
        pass
        
    else:
        Product_Link.append(product_link)
        Product_Brand.append(product_brand)
        Product_Name.append(product_name)
        Description.append(product_des)
        Gender.append('Men')
        Cat_list.append(product_cat)
        MRP.append(product_mrp)
        Price.append(product_price)
        Primary_Image_Link.append(primary_link)
        Secondary_Image_Link.append(secondary_link)
        Size_Available.append(size_avl)
    

100%|██████████| 64/64 [01:20<00:00,  1.26s/it]


In [34]:
out=pd.DataFrame({
    
    "Website":'https://apparels.azanisport.com/',
    "Product_Link":Product_Link,
    "Product_Name":Product_Name,
    "Product_Brand":Product_Brand,
    "Product_Category":Cat_list,
    "Size_Avail":Size_Available,
    "Price":Price,
    "MRP":MRP,
    "Gender":Gender,
    "Description":Description,
    "Primary_Image_Links":Primary_Image_Link,
    "Secondary_Image_Links":Secondary_Image_Link,
    "Affiliate_Link":''
    
})

In [35]:
out.head()

,Website,Product_Link,Product_Name,Product_Brand,Product_Category,Size_Avail,Price,MRP,Gender,Description,Primary_Image_Links,Secondary_Image_Links,Affiliate_Link
0,https://apparels.azanisport.com/,https://apparels.azanisport.com/collections/me...,Azani Super Spirit Sleeveless Compression Top ...,Azani,Compression Top,"S,M,L,XL,XXL",688,1899,Men,Product Description:-\nThe Azani Super Spirit ...,https://cdn.shopify.com/s/files/1/0910/2946/pr...,//cdn.shopify.com/s/files/1/0910/2946/products...,
1,https://apparels.azanisport.com/,https://apparels.azanisport.com/collections/me...,Azani Men’s Pixel 1/4 Zipper Jumper - Grey,Azani,Jumper,"S,M,L,XL,XXL",1337,1999,Men,Product Description:\nThe Azani Men’s Pixel 1/...,https://cdn.shopify.com/s/files/1/0910/2946/pr...,//cdn.shopify.com/s/files/1/0910/2946/products...,
2,https://apparels.azanisport.com/,https://apparels.azanisport.com/collections/me...,Azani Dart Crew Neck Half Sleeve T-Shirt - Yellow,Azani,T-Shirt,"S,M,L,XL,XXL",340,1200,Men,Product Description:\nA simple yet sophisticat...,https://cdn.shopify.com/s/files/1/0910/2946/pr...,//cdn.shopify.com/s/files/1/0910/2946/products...,
3,https://apparels.azanisport.com/,https://apparels.azanisport.com/collections/me...,Azani Sprint Light Weight Woven Jacket - Teal/...,Azani,Jacket,"S,M,L,XL,XXL",899,1999,Men,Product Description:\nThe Azani Sprint Light W...,https://cdn.shopify.com/s/files/1/0910/2946/pr...,//cdn.shopify.com/s/files/1/0910/2946/products...,
4,https://apparels.azanisport.com/,https://apparels.azanisport.com/collections/me...,Azani Tapered Track Pants - Navy,Azani,Track Pants,"XS,S,M,L,XL,XXL",519,1299,Men,"Product Description:\nSleek, lightweight and f...",https://cdn.shopify.com/s/files/1/0910/2946/pr...,//cdn.shopify.com/s/files/1/0910/2946/products...,


In [36]:
conn = sqlite3.connect(r'azani.db')
c = conn.cursor()

c.execute('CREATE TABLE product_details (Website varchar(40) not null,Product_Link text PRIMARY KEY,Product_Name varchar(50) not null,Product_Brand varchar(50) not null,Product_Category varchar(50),Size_Avail varchar(20) not null,Price int not null,MRP int not null,Gender varchar(15) not null,Description text not null,Primary_Image_Links text not null,Secondary_Image_Links text not null,Affiliate_Link text not null )')
conn.commit()

out.to_sql('product_details', conn, if_exists='replace', index = False)
conn.close()

## Mens Shoes

In [37]:
shoes_link=[]
for i in range(1,2):
    try:
        result=requests.get(f"https://apparels.azanisport.com/collections/mens-shoes?page={i}",headers=headers)
        get=result.content
        data=BeautifulSoup(get,'lxml')
        prod=data.find_all('div',{'class':'Grid__Cell 1/2--phone 1/2--tablet-and-up 1/3--desk'})
        for j in prod:
            if(j.find('button').find('span').text !='SOLD OUT'):
                shoes_link.append('https://apparels.azanisport.com'+j.find('a').attrs['href'])
    except:
        pass

In [38]:
len(shoes_link)

29

In [39]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
Gender=[]
Description=[]
Cat_list=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [40]:
for i in tqdm(shoes_link):
    try:
        result=requests.get(i,headers=headers)
        get=result.content
        soup=BeautifulSoup(get,'lxml')
        js=json.loads(soup.find('script',{'type':'application/ld+json'}).contents[0])
        product_name=js['name']
        product_brand=js['brand']
        product_link=i
        product_cat=js['category']
        product_des=js['description']
        product_mrp=soup.find('span',{'class':'ProductMeta__Price Price Price--compareAt Text--subdued u-h4'}).text
        product_mrp=int(''.join(product_mrp[4:].split(',')))
        product_price=int(float(js['offers']['price']))
        primary_link=js['image']['url']
        images=soup.find_all('div',{'class':'Product__SlideItem Product__SlideItem--image Carousel__Cell'})
        img=[]
        for k in images:
            img.append(k.find('img').attrs['data-src'].split('{width}')[0]+'1024x1024.jpg')
        secondary_link=','.join(img)
        size=soup.find('ul',{'class':'SizeSwatchList HorizontalList HorizontalList--spacingTight'}).text.lstrip().rstrip().split('\n\n\n')
        size_avl=','.join(size)
    
    except:
        pass
        
    else:
        Product_Link.append(product_link)
        Product_Brand.append(product_brand)
        Product_Name.append(product_name)
        Description.append(product_des)
        Gender.append('Men')
        Cat_list.append(product_cat)
        MRP.append(product_mrp)
        Price.append(product_price)
        Primary_Image_Link.append(primary_link)
        Secondary_Image_Link.append(secondary_link)
        Size_Available.append(size_avl)
    

100%|██████████| 29/29 [00:36<00:00,  1.25s/it]


In [41]:
out=pd.DataFrame({
    
    "Website":'https://apparels.azanisport.com/',
    "Product_Link":Product_Link,
    "Product_Name":Product_Name,
    "Product_Brand":Product_Brand,
    "Product_Category":Cat_list,
    "Size_Avail":Size_Available,
    "Price":Price,
    "MRP":MRP,
    "Gender":Gender,
    "Description":Description,
    "Primary_Image_Links":Primary_Image_Link,
    "Secondary_Image_Links":Secondary_Image_Link,
    "Affiliate_Link":''
    
})

In [42]:
out.shape

(29, 13)

In [43]:
conn = sqlite3.connect(r'azani.db')
c = conn.cursor()

#c.execute('CREATE TABLE product_details (Website varchar(40) not null,Product_Link text PRIMARY KEY,Product_Name varchar(50) not null,Product_Brand varchar(50) not null,Product_Category varchar(50),Size_Avail varchar(20) not null,Price int not null,MRP int not null,Gender varchar(15) not null,Description text not null,Primary_Image_Links text not null,Secondary_Image_Links text not null,Affiliate_Link text not null )')
#conn.commit()

out.to_sql('product_details', conn, if_exists='append', index = False)
conn.close()

## Mens Accessories

In [44]:
acc_link=[]
for i in range(1,2):
    try:
        result=requests.get(f"https://apparels.azanisport.com/collections/mens-trackpants-joggers?page={i}",headers=headers)
        get=result.content
        data=BeautifulSoup(get,'lxml')
        prod=data.find_all('div',{'class':'Grid__Cell 1/2--phone 1/2--tablet-and-up 1/3--desk'})
        for j in prod:
            if(j.find('button').find('span').text !='SOLD OUT'):
                acc_link.append('https://apparels.azanisport.com'+j.find('a').attrs['href'])
    except:
        pass

In [45]:
len(acc_link)

9

In [46]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
Gender=[]
Description=[]
Cat_list=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [47]:
for i in tqdm(acc_link):
    try:
        result=requests.get(i,headers=headers)
        get=result.content
        soup=BeautifulSoup(get,'lxml')
        js=json.loads(soup.find('script',{'type':'application/ld+json'}).contents[0])
        product_name=js['name']
        product_brand=js['brand']
        product_link=i
        product_cat=js['category']
        product_des=js['description']
        product_mrp=soup.find('span',{'class':'ProductMeta__Price Price Price--compareAt Text--subdued u-h4'}).text
        product_mrp=int(''.join(product_mrp[4:].split(',')))
        product_price=int(float(js['offers']['price']))
        primary_link=js['image']['url']
        images=soup.find_all('div',{'class':'Product__SlideItem Product__SlideItem--image Carousel__Cell'})
        img=[]
        for k in images:
            img.append(k.find('img').attrs['data-src'].split('{width}')[0]+'1024x1024.jpg')
        secondary_link=','.join(img)
        size=soup.find('ul',{'class':'SizeSwatchList HorizontalList HorizontalList--spacingTight'}).text.lstrip().rstrip().split('\n\n\n')
        size_avl=','.join(size)
    
    except:
        pass
        
    else:
        Product_Link.append(product_link)
        Product_Brand.append(product_brand)
        Product_Name.append(product_name)
        Description.append(product_des)
        Gender.append('Men')
        Cat_list.append(product_cat)
        MRP.append(product_mrp)
        Price.append(product_price)
        Primary_Image_Link.append(primary_link)
        Secondary_Image_Link.append(secondary_link)
        Size_Available.append(size_avl)
    

100%|██████████| 9/9 [00:08<00:00,  1.07it/s]


In [48]:
out=pd.DataFrame({
    
    "Website":'https://apparels.azanisport.com/',
    "Product_Link":Product_Link,
    "Product_Name":Product_Name,
    "Product_Brand":Product_Brand,
    "Product_Category":Cat_list,
    "Size_Avail":Size_Available,
    "Price":Price,
    "MRP":MRP,
    "Gender":Gender,
    "Description":Description,
    "Primary_Image_Links":Primary_Image_Link,
    "Secondary_Image_Links":Secondary_Image_Link,
    "Affiliate_Link":''
    
})

In [49]:
out.shape

(9, 13)

In [50]:
conn = sqlite3.connect(r'azani.db')
c = conn.cursor()

#c.execute('CREATE TABLE product_details (Website varchar(40) not null,Product_Link text PRIMARY KEY,Product_Name varchar(50) not null,Product_Brand varchar(50) not null,Product_Category varchar(50),Size_Avail varchar(20) not null,Price int not null,MRP int not null,Gender varchar(15) not null,Description text not null,Primary_Image_Links text not null,Secondary_Image_Links text not null,Affiliate_Link text not null )')
#conn.commit()

out.to_sql('product_details', conn, if_exists='append', index = False)
conn.close()